In [13]:
import rope.base.project
from rope.refactor.extract import ExtractMethod
import re

In [38]:

APOSTROPHE_MARKER = "__APOSTROPHE__"

def serialize(text: str):
    # Replace "'{var}'" with "__APOSTROPHE__{var}__APOSTROPHE__"
    return re.sub(r"'(.*?)'", f"{APOSTROPHE_MARKER}\\1{APOSTROPHE_MARKER}", text)

def deserialize(text: str):
    return re.sub(f"{APOSTROPHE_MARKER}(.*?){APOSTROPHE_MARKER}", "'\\1'", text)

myproject = rope.base.project.Project('src')

myresource = myproject.get_resource('test.py')
contents = myresource.read()
serialized_contents = serialize(myresource.read())
myresource.write(serialized_contents)
extract_span = r'''    single_payment_link = "https://buy.stripe.com/00g3fh7qF85q0AE14d"
    pro_payment_link = "https://buy.stripe.com/00g5npeT71H2gzCfZ8"
    daily_message = (
        f" and {daily_ticket_count} for the day"
        if not is_paying_user and not is_consumer_tier
        else ""
    )
    user_type = (
        "💎 <b>Sweep Pro</b>" if is_paying_user else "⚡ <b>Sweep Basic Tier</b>"
    )
    gpt_tickets_left_message = (
        f"{ticket_count} GPT-4 tickets left for the month"
        if not is_paying_user
        else "unlimited GPT-4 tickets"
    )
    purchase_message = f"<br/><br/> For more GPT-4 tickets, visit <a href=[{single_payment_link}]>our payment portal</a>. For a one week free trial, try <a href='{pro_payment_link}'>Sweep Pro</a> (unlimited GPT-4 tickets)."'''
serialized_extract_span = serialize(extract_span)

start, end = serialized_contents.find(serialized_extract_span), serialized_contents.find(serialized_extract_span) + len(serialized_extract_span)
print(start, end)

try:
    extractor = ExtractMethod(myproject, myresource, start, end)
    change_set = extractor.get_changes("helper")
    for change in change_set.changes:
        if change.old_contents is not None:
            change.old_contents = deserialize(change.old_contents)
        else:
            change.old_contents = deserialize(change.resource.read())
        change.new_contents = deserialize(change.new_contents)
    for change in change_set.changes:
        print(change.get_description())
except Exception as e:
    print(e)
finally:
    myresource.write(contents)

299 1104
--- a/test.py
+++ b/test.py
@@ -9,6 +9,10 @@
     is_consumer_tier = False
     model_name = "GPT-3.5" if use_faster_model else "GPT-4"
     payment_link = "https://sweep.dev/pricing"
+    helper(is_paying_user, is_consumer_tier, daily_ticket_count, ticket_count)
+    print(b, c)
+
+def helper(is_paying_user, is_consumer_tier, daily_ticket_count, ticket_count):
     single_payment_link = "https://buy.stripe.com/00g3fh7qF85q0AE14d"
     pro_payment_link = "https://buy.stripe.com/00g5npeT71H2gzCfZ8"
     daily_message = (
@@ -25,4 +29,3 @@
         else "unlimited GPT-4 tickets"
     )
     purchase_message = f"<br/><br/> For more GPT-4 tickets, visit <a href=[{single_payment_link}]>our payment portal</a>. For a one week free trial, try <a href='{pro_payment_link}'>Sweep Pro</a> (unlimited GPT-4 tickets)."
-    print(b, c)



In [37]:
def extract_method(
    snippet,
    file_path,
    method_name,
    project_name="../../sweepai"
):
    project = rope.base.project.Project(project_name)

    resource = project.get_resource(file_path)
    contents = resource.read()
    start, end = contents.find(snippet), contents.find(snippet) + len(snippet)

    extractor = ExtractMethod(project, resource, start, end)
    change_set = extractor.get_changes(method_name, similar=True, global_=True)
    for change in change_set.changes:
        change.do()

    result = resource.read()

    for change in change_set.changes:
        change.undo()
    return result

snippet = r"""edit_sweep_comment(
                review_message + "\n\nSuccess! 🚀",
                4,
                pr_message=(
                    f"## Here's the PR! [{pr.html_url}]({pr.html_url}).\n{center(payment_message_start)}"
                ),
                done=True,
            )"""
print(extract_method(snippet, "handlers/on_ticket.py", "hydrate_sandbox"))

SyntaxError: f-string: invalid syntax (<string>, line 5)